In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
from category_encoders.target_encoder import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (GridSearchCV,KFold, train_test_split)
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix

In [2]:
import warnings

# Para ignorar todos os warnings:
warnings.filterwarnings('ignore')

# Simulado

In [3]:
dataset_teste = pd.read_csv('dataset_teste.csv')

In [52]:
import joblib
# Carregar o modelo treinado
modelo_carregado = joblib.load('modelo_treinado.joblib')

In [5]:
dataset_treino = pd.read_csv('dataset_treinamento.csv')

In [6]:
X_test = dataset_teste.drop(['tp_performance_entrega'],axis=1)
y_test = dataset_teste[['tp_performance_entrega']]

In [7]:
y_test.sum()

tp_performance_entrega    3548
dtype: int64

In [8]:
len(y_test)

93503

In [49]:
3548/93503

0.03794530656770371

In [73]:
(3548*0.04)/93503

0.0015178122627081485

In [10]:
columns_to_encoder=['uf', 'tp_praca', 'des_unidade_negocio', 'des_unidade_negocio','uf_origem', 'grp_transportadora']

In [11]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first')

encoded_data = encoder.fit_transform(X_test[columns_to_encoder])

encoded_feature_names = encoder.get_feature_names_out(input_features=columns_to_encoder)

encoded_df = pd.DataFrame(encoded_data, columns=encoded_feature_names)

In [12]:
X_test_encoded = X_test.drop(columns_to_encoder, axis=1)

In [13]:
df_encoded = pd.concat([X_test_encoded, encoded_df], axis=1)

In [14]:
X_test.columns

Index(['uf', 'grp_transportadora', 'tp_praca', 'des_unidade_negocio',
       'uf_origem', 'intervalo_pedido_pagamento_s', 'indice_mes_s',
       'status_rota_transp_s', 'percentual_viagens_rota_s'],
      dtype='object')

In [15]:
y_pred = modelo_carregado.predict(X_test)

In [16]:
y_pred = pd.DataFrame(y_pred, columns = ['predict'])

In [17]:
len(list(filter(lambda x: x == 1, y_pred.predict)))

74795

In [55]:
0.04*74795

2991.8

In [56]:
2991.8/93503

0.031996834326171356

In [20]:
df_merge = pd.read_csv('grp_transportadora.csv')

In [68]:
(2991-108)/2991

0.9638916750250752

In [21]:
lista_simulacao = []
for index, row in X_test.iterrows():
    estado_origem = row['uf_origem'] 
    estado_destino = row['uf']
    lista_transportadoras_trecho = df_merge[(df_merge['uf_origem'] == estado_origem) & (df_merge['uf'] == estado_destino)] 
    for transportadora in lista_transportadoras_trecho.grp_transportadora:
        lista_simulacao.append([index,
            row['uf'], row['tp_praca'], row['des_unidade_negocio'],
            row['intervalo_pedido_pagamento_s'], row['uf_origem'], transportadora,
            row['indice_mes_s']
        ])

In [22]:
df_temp = pd.DataFrame(lista_simulacao, columns = ['codigo_unico', 'uf', 'tp_praca', 'des_unidade_negocio',
            'intervalo_pedido_pagamento_s', 'uf_origem', 'grp_transportadora',
            'indice_mes_s'])

In [23]:
df_simulado = pd.merge(df_temp, df_merge, on=['grp_transportadora', 'uf', 'uf_origem'])

In [24]:
min_max_params = pd.read_csv('min_max_params.csv')
min_values = min_max_params['min'].values
max_values = min_max_params['max'].values

scaler = MinMaxScaler(feature_range=(min_values[2], max_values[2]))
normalized_data = scaler.fit_transform(df_simulado[['status_rota_transp']])
name_column = 'status_rota_transp' + '_s'
df_simulado[name_column] = normalized_data

scaler = MinMaxScaler(feature_range=(min_values[3], max_values[3]))
normalized_data = scaler.fit_transform(df_simulado[['percentual_viagens_rota']])
name_column = 'percentual_viagens_rota' + '_s'
df_simulado[name_column] = normalized_data

In [25]:
df_simulado.head()

,codigo_unico,uf,tp_praca,des_unidade_negocio,intervalo_pedido_pagamento_s,uf_origem,grp_transportadora,indice_mes_s,viagens,viagens_atrasadas_rota_transp,status_rota_transp,percentual_atraso,total_viagens_transp,percentual_viagens_rota,status_transportadora,status_rota_transp_s,percentual_viagens_rota_s
0,0,RS,Interior,Mono,0.791667,PR,Transportadora 2,0.382516,549,72,1.462659,0.131148,57658,0.009522,1.82332,1.462659,0.009522
1,4,RS,Reg. Metropolitana,Mono,0.791667,PR,Transportadora 2,0.806398,549,72,1.462659,0.131148,57658,0.009522,1.82332,1.462659,0.009522
2,8,RS,Capital,Mono,0.791667,PR,Transportadora 2,0.000000,549,72,1.462659,0.131148,57658,0.009522,1.82332,1.462659,0.009522
3,27,RS,Reg. Metropolitana,Multi,0.791667,PR,Transportadora 2,0.806398,549,72,1.462659,0.131148,57658,0.009522,1.82332,1.462659,0.009522
4,37,RS,Interior,Mono,0.791667,PR,Transportadora 2,0.382516,549,72,1.462659,0.131148,57658,0.009522,1.82332,1.462659,0.009522


In [26]:
df_simulado.columns

Index(['codigo_unico', 'uf', 'tp_praca', 'des_unidade_negocio',
       'intervalo_pedido_pagamento_s', 'uf_origem', 'grp_transportadora',
       'indice_mes_s', 'viagens', 'viagens_atrasadas_rota_transp',
       'status_rota_transp', 'percentual_atraso', 'total_viagens_transp',
       'percentual_viagens_rota', 'status_transportadora',
       'status_rota_transp_s', 'percentual_viagens_rota_s'],
      dtype='object')

In [27]:
df_simulado = df_simulado.drop(['viagens', 'viagens_atrasadas_rota_transp',
       'status_rota_transp', 'percentual_atraso', 'total_viagens_transp',
       'percentual_viagens_rota', 'status_transportadora',], axis=1)

In [28]:
df_simulado.columns

Index(['codigo_unico', 'uf', 'tp_praca', 'des_unidade_negocio',
       'intervalo_pedido_pagamento_s', 'uf_origem', 'grp_transportadora',
       'indice_mes_s', 'status_rota_transp_s', 'percentual_viagens_rota_s'],
      dtype='object')

In [29]:
df_simulado[(df_simulado['codigo_unico'] == 4)]

,codigo_unico,uf,tp_praca,des_unidade_negocio,intervalo_pedido_pagamento_s,uf_origem,grp_transportadora,indice_mes_s,status_rota_transp_s,percentual_viagens_rota_s
1,4,RS,Reg. Metropolitana,Mono,0.791667,PR,Transportadora 2,0.806398,1.462659,0.009522
4934,4,RS,Reg. Metropolitana,Mono,0.791667,PR,Transportadora 3,0.806398,1.913043,0.000229
9867,4,RS,Reg. Metropolitana,Mono,0.791667,PR,Transportadora 7,0.806398,2.000000,0.001666
14800,4,RS,Reg. Metropolitana,Mono,0.791667,PR,Transportadora 4,0.806398,1.719854,0.286453


In [30]:
df_filtrado = df_simulado.loc[df_simulado.groupby(['uf', 'uf_origem', 'codigo_unico'])['status_rota_transp_s'].idxmax()].reset_index(drop=True)

In [31]:
df_filtrado[(df_filtrado['codigo_unico'] == 4)]

,codigo_unico,uf,tp_praca,des_unidade_negocio,intervalo_pedido_pagamento_s,uf_origem,grp_transportadora,indice_mes_s,status_rota_transp_s,percentual_viagens_rota_s
48688,4,RS,Reg. Metropolitana,Mono,0.791667,PR,Transportadora 7,0.806398,2.0,0.001666


In [32]:
len(df_filtrado)

93503

In [33]:
len(y_test)

93503

In [34]:
y_test.reset_index(inplace=True)

In [35]:
y_test.columns = ['codigo_unico', 'real']

In [36]:
df = pd.merge(df_filtrado, y_test, on=['codigo_unico'])

In [37]:
df

,codigo_unico,uf,tp_praca,des_unidade_negocio,intervalo_pedido_pagamento_s,uf_origem,grp_transportadora,indice_mes_s,status_rota_transp_s,percentual_viagens_rota_s,real
0,2905,AC,Interior,Multi,0.777778,PR,Transportadora 2,0.989592,2.0,0.000104,0
1,2990,AC,Capital,Mono,0.791667,PR,Transportadora 2,0.382516,2.0,0.000104,0
2,3168,AC,Interior,Multi,0.791667,PR,Transportadora 2,0.806398,2.0,0.000104,0
3,5039,AC,Interior,Multi,0.791667,PR,Transportadora 2,0.711235,2.0,0.000104,0
4,5624,AC,Capital,Multi,0.777778,PR,Transportadora 2,1.000000,2.0,0.000104,1
...,...,...,...,...,...,...,...,...,...,...,...
93498,87502,TO,Interior,Mono,0.791667,SP,Transportadora 1,0.711235,2.0,0.000447,0
93499,88558,TO,Capital,Mono,0.791667,SP,Transportadora 1,1.000000,2.0,0.000447,0
93500,89884,TO,Interior,Multi,0.791667,SP,Transportadora 1,0.382516,2.0,0.000447,0
93501,90029,TO,Interior,Mono,0.791667,SP,Transportadora 1,0.711235,2.0,0.000447,0


In [38]:
y_pred = modelo_carregado.predict(df)

In [39]:
y_pred = pd.DataFrame(y_pred, columns = ['predict'])

In [40]:
df_simulado_predict = pd.concat([df, y_pred], axis=1)

In [41]:
df_simulado_predict

,codigo_unico,uf,tp_praca,des_unidade_negocio,intervalo_pedido_pagamento_s,uf_origem,grp_transportadora,indice_mes_s,status_rota_transp_s,percentual_viagens_rota_s,real,predict
0,2905,AC,Interior,Multi,0.777778,PR,Transportadora 2,0.989592,2.0,0.000104,0,0
1,2990,AC,Capital,Mono,0.791667,PR,Transportadora 2,0.382516,2.0,0.000104,0,0
2,3168,AC,Interior,Multi,0.791667,PR,Transportadora 2,0.806398,2.0,0.000104,0,0
3,5039,AC,Interior,Multi,0.791667,PR,Transportadora 2,0.711235,2.0,0.000104,0,0
4,5624,AC,Capital,Multi,0.777778,PR,Transportadora 2,1.000000,2.0,0.000104,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
93498,87502,TO,Interior,Mono,0.791667,SP,Transportadora 1,0.711235,2.0,0.000447,0,0
93499,88558,TO,Capital,Mono,0.791667,SP,Transportadora 1,1.000000,2.0,0.000447,0,0
93500,89884,TO,Interior,Multi,0.791667,SP,Transportadora 1,0.382516,2.0,0.000447,0,0
93501,90029,TO,Interior,Mono,0.791667,SP,Transportadora 1,0.711235,2.0,0.000447,0,0


In [42]:
df_simulado_predict['predict'].sum()

2704

In [57]:
2704 * 0.04

108.16

In [59]:
108.16/93503*100

0.11567543287381153